# Question 1

In [2]:
import numpy as np
import pandas as pd

# Needleman-Wunsch algorithm for global sequence alignment.
def nw(x, y, match=2, mismatch=-3, gap=-1):
    nx = len(x)
    ny = len(y)

    # Initialization of the matrix.
    F = np.zeros((nx + 1, ny + 1))
    F[:, 0] = np.linspace(0, gap * nx, nx + 1)
    F[0, :] = np.linspace(0, gap * ny, ny + 1)

    # Pointers to trace through an optimal alignment.
    P = np.zeros((nx + 1, ny + 1), dtype=int)
    P[:, 0] = 3
    P[0, :] = 4

    # Matrix filling.
    for i in range(1, nx + 1):
        for j in range(1, ny + 1):
            match_score = match if x[i - 1] == y[j - 1] else mismatch
            a1 = F[i - 1, j - 1] + match_score
            a2 = F[i - 1, j] + gap 
            a3 = F[i, j - 1] + gap
            F[i, j] = max(a1, a2, a3)
            if (a1 == F[i, j]): # Match or mismatch
                P[i, j] += 2
            if (a2 == F[i, j]): # Gap in one of the sequence 
                P[i, j] += 3
            if (a3 == F[i, j]): # Gap in the other sequence
                P[i, j] += 4

    # Print scoring matrix using pandas DataFrame
    print("Scoring Matrix:")
    df = pd.DataFrame(F, index=['-'] + list(x), columns=['-'] + list(y))
    print(df)

    # Iterative traceback
    alignments = []
    stack = [(nx, ny, '', '', 0)]
    # Using stack to avoid recursion depth limit.
    # Each element in the stack is a tuple (i, j, rx, ry, score).
    # i, j: current position in the matrix.
    # rx, ry: partial alignments.
    # score: current alignment score.
    # The stack is initialized with the last element of the matrix. Last element is the bottom right corner of the matrix.
    # The traceback is performed iteratively until the stack is empty.
    # The traceback is performed in a depth-first manner.
    # The traceback is finished when the current position is (0, 0) and current score is 0.
    while stack:
        i, j, rx, ry, score = stack.pop()
        if i == 0 and j == 0:
            alignments.append((rx[::-1], ry[::-1], score))
        else:
            if P[i, j] in [2, 5, 6, 9]:
                if x[i - 1] == y[j - 1]:
                    stack.append((i - 1, j - 1, rx + x[i - 1], ry + y[j - 1], score + match))
                else:
                    stack.append((i - 1, j - 1, rx + x[i - 1], ry + y[j - 1], score + mismatch))
            if P[i, j] in [3, 5, 7, 9]:
                stack.append((i - 1, j, rx + x[i - 1], ry + '-', score + gap))
            if P[i, j] in [4, 6, 7, 9]:
                stack.append((i, j - 1, rx + '-', ry + y[j - 1], score + gap))

    # Print all optimal alignments.
    print("\nAll Optimal Global Alignments:\n")
    for alignment in alignments: # this will print the complete alignment end to end
        print('\n'.join(alignment[:2]))
        print("Score:", alignment[2])
        print()

    return

seq1 = "GATGCGCAG"
seq2 = "GGCAGTA"

nw(seq1, seq2)

Scoring Matrix:
     -    G    G    C    A    G    T    A
-  0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0
G -1.0  2.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0
A -2.0  1.0  0.0 -1.0  2.0  1.0  0.0 -1.0
T -3.0  0.0 -1.0 -2.0  1.0  0.0  3.0  2.0
G -4.0 -1.0  2.0  1.0  0.0  3.0  2.0  1.0
C -5.0 -2.0  1.0  4.0  3.0  2.0  1.0  0.0
G -6.0 -3.0  0.0  3.0  2.0  5.0  4.0  3.0
C -7.0 -4.0 -1.0  2.0  1.0  4.0  3.0  2.0
A -8.0 -5.0 -2.0  1.0  4.0  3.0  2.0  5.0
G -9.0 -6.0 -3.0  0.0  3.0  6.0  5.0  4.0

All Optimal Global Alignments:

GATGCGCAG--
G--GC--AGTA
Score: 4

GATGCGCAG--
G--G--CAGTA
Score: 4

GATGCGCAG--
G----GCAGTA
Score: 4

GATGCGCAG--
---G-GCAGTA
Score: 4

GATGC-GC-AG
G--GCAG-TA-
Score: 4

GATGC-G-CAG
G--GCAGT-A-
Score: 4



# Question 2

In [1]:
import numpy as np
import pandas as pd

# Smith-Waterman algorithm for local sequence alignment. 
def sw(x, y, match=2, mismatch=-1, gap=-3):
    nx = len(x)
    ny = len(y)

    # Initialization of the matrix.
    F = np.zeros((nx + 1, ny + 1))

    # Pointers to trace through an optimal alignment.
    P = np.zeros((nx + 1, ny + 1), dtype=int)
    P[:, 0] = 3
    P[0, :] = 4

    max_score = 0
    max_i = 0
    max_j = 0

    # Matrix filling.
    max_positions = []
    for i in range(1, nx + 1):
        for j in range(1, ny + 1):
            match_score = match if x[i - 1] == y[j - 1] else mismatch
            a1 = F[i - 1, j - 1] + match_score
            a2 = F[i - 1, j] + gap
            a3 = F[i, j - 1] + gap
            F[i, j] = max(a1, a2, a3, 0)
            if (a1 == F[i, j]): # Match or mismatch
                P[i, j] += 2
            if (a2 == F[i, j]): # Gap in one of the sequence
                P[i, j] += 3
            if (a3 == F[i, j]): # Gap in the other sequence
                P[i, j] += 4

            if F[i, j] > max_score:
                max_score = F[i, j]
                max_positions = [(i, j)]
            elif F[i, j] == max_score:
                max_positions.append((i, j))

    # Print scoring matrix using pandas DataFrame
    print("Scoring Matrix:")
    df = pd.DataFrame(F, index=['-'] + list(x), columns=['-'] + list(y))
    print(df)

    # Iterative traceback
    alignments = []
    for max_i, max_j in max_positions:
        stack = [(max_i, max_j, '', '', 0)]
        # Using stack to avoid recursion depth limit.
        # Each element in the stack is a tuple (i, j, rx, ry, score).
        # i, j: current position in the matrix.
        # rx, ry: partial alignments.
        # score: current alignment score.
        # The stack is initialized with one of the maximum scores in the matrix.
        # The traceback is performed iteratively until the stack is empty.
        # The traceback is performed in a depth-first manner.
        # The traceback is finished when the current score is 0.
        while stack:
            i, j, rx, ry, score = stack.pop()
            if F[i, j] == 0:
                alignments.append((rx[::-1], ry[::-1], score))
            else:
                if P[i, j] in [2, 5, 6, 9]:
                    if x[i - 1] == y[j - 1]:
                        stack.append((i - 1, j - 1, rx + x[i - 1], ry + y[j - 1], score + match))
                    else:
                        stack.append((i - 1, j - 1, rx + x[i - 1], ry + y[j - 1], score + mismatch))
                if P[i, j] in [3, 5, 7, 9]:
                    stack.append((i - 1, j, rx + x[i - 1], ry + '-', score + gap))
                if P[i, j] in [4, 6, 7, 9]:
                    stack.append((i, j - 1, rx + '-', ry + y[j - 1], score + gap))

    # Print all optimal alignments.
    print("\nAll Optimal Local Alignments:\n")
    for alignment in alignments: # this will print the motif of the alignment which is common in both
        print('\n'.join(alignment[:2]))
        print("Score:", alignment[2])
        print()

    return

seq1 = "GATGCGCAG"
seq2 = "GGCAGTA"

sw(seq1, seq2)

Scoring Matrix:
     -    G    G    C    A    G    T    A
-  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
G  0.0  2.0  2.0  0.0  0.0  2.0  0.0  0.0
A  0.0  0.0  1.0  1.0  2.0  0.0  1.0  2.0
T  0.0  0.0  0.0  0.0  0.0  1.0  2.0  0.0
G  0.0  2.0  2.0  0.0  0.0  2.0  0.0  1.0
C  0.0  0.0  1.0  4.0  1.0  0.0  1.0  0.0
G  0.0  2.0  2.0  1.0  3.0  3.0  0.0  0.0
C  0.0  0.0  1.0  4.0  1.0  2.0  2.0  0.0
A  0.0  0.0  0.0  1.0  6.0  3.0  1.0  4.0
G  0.0  2.0  2.0  0.0  3.0  8.0  5.0  2.0

All Optimal Local Alignments:

GCAG
GCAG
Score: 8



C:\Users\Sameer-Singh-Godara\AppData\Local\Temp\ipykernel_19672\460279793.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
